# Play ground for Engeenering the Prompt

In [1]:
import os
import openai
import pandas as pd
import query_data
import datetime as dt
import pytz
from langchain.llms import OpenAIChat
import configparser
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
config = configparser.ConfigParser()
config.read('config.ini')
openai.api_key = config['openai']['api_key']
os.environ["OPENAI_API_KEY"] = config['openai']['api_key']


In [2]:
llm = OpenAIChat(temperature=0)

## Simple summary

In [54]:

prompt_narrative_crypto = """Create a list of crypto $TOKEN from the text and follow this guide:
                             1) Tokens mentioned in the tweets should be in the format $TOKEN
                             2) Unify information for each $TOKEN.
                             3) For each $TOKEN, inclued narrative  or relevant information.
                             4) Add catalysts for each token if possible.
                             5) if No specific $TOKEN mentioned, include a general narrative.
    response example: "1) $BTC:  Posible pump due to Elon Musk's tweet. Catalyst: Next convention in march.
                       2) $HPK:  Strong narrative  Catalyst: No info."



   {text}
   """


In [55]:
# load tweets
tweets = pd.read_pickle("tweets_crypto.pkl")
# get tweets from last 24 hours
last_12 = dt.datetime.now(pytz.UTC) - dt.timedelta(hours=24)
tweets = [ tweet for tweet in tweets  if tweet.created_at >= last_12]
docs = query_data.create_thread_docs(tweets, filer_len=2)

In [56]:


PROMPT = PromptTemplate(template=prompt_narrative_crypto, input_variables=["text"])
chain = load_summarize_chain(llm, chain_type="stuff", prompt=PROMPT)
chain.run(docs)

'\n\n1) $ETH: Currently experiencing volatility and flailing around under $1600. Catalyst: Shanghai upgrade causing amplified sell pressure.\n2) $BTC: Experienced a 5% drop in price, causing concern among investors and traders. Catalysts: Mt. Gox unlock, mass liquidations, regulatory crypto clampdown.\n3) $RPL: Old favorite with little chatter on the TL. Picked up at a low price, but closed in a small profit. Catalyst: No info.\n4) $STX: Took a little off the table and played outside perfectly. Catalyst: Magic Eden free NFT mints.\n5) General narrative: Importance of balancing all aspects of life, including wellness and trading. Catalyst: None mentioned.'

## Combine summary win tokens more than 4000

In [57]:
refine_template = (
        "your job is to create a list of crypto $TOKEN from the text and follow this guide:\n"
                             "1) Tokens mentioned in the tweets should be in the format $TOKEN\n"
                             "2) Unify information for each $TOKEN.\n"
                             "3) For each $TOKEN, inclued narrative  or relevant information.\n"
                             "4) Add catalysts for each token if possible.\n"
                             "5) if No specific $TOKEN mentioned, include a general narrative.\n"
        "We have provided an existing respond up to a certain point: {existing_answer}\n"
        "We have the opportunity to refine the existing list of $TOKEN \n"
        "(only if needed) with some more text below.\n"
        "------------\n"
        "{text}\n"
        "------------\n"
        "Given the new text, refine the original list"
        "If the text isn't useful, return the original list."
    )
refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)
chain = load_summarize_chain(llm, chain_type="refine", return_intermediate_steps=True,
                             question_prompt=PROMPT, refine_prompt=refine_prompt)
chain({"input_documents": docs}, return_only_outputs=True)['output_text']


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-ujTxfa4kohWwQMLMLnUCZg8A on requests per min. Limit: 20 / min. Current: 30 / min. Contact support@openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-ujTxfa4kohWwQMLMLnUCZg8A on requests per min. Limit: 20 / min. Current: 30 / min. Contact support@openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.c

KeyboardInterrupt: 